<div Style="text-align: center;line-height: 30px;font-size:24px;font-weight: bold"> Etiquetador de imágenes</div>
***

In [290]:
from __future__ import print_function
#Imports
%matplotlib inline
import numpy as np

In [291]:
import ipywidgets as widgets
from traitlets import Unicode, validate, Bool, Any, List, observe
from ipywidgets import Color
from skimage import io
from skimage.transform import resize

class ImageLabelerWidget(widgets.DOMWidget):
    _view_name = Unicode('ImageLabelerView').tag(sync=True)
    _view_module = Unicode('ImageLabeler').tag(sync=True)
    # Atributos sincronos entre js y python
    image = Unicode('blank.png').tag(sync=True)
    rect = List().tag(sync=True)
    
    Phytolith_type = Any().tag(sync=True)
    
    def __init__(self, *pargs, **kwargs):
        super(widgets.DOMWidget, self).__init__(*pargs, **kwargs)
        self._rectList = []
        
    @observe('rect')
    def new_rect(self, change):
        rec = [round(coord) for coord in change["new"]]
        self._rectList.append(rec)
        self.save_coords_as_images()
        
    def save_coords_as_images(self):
        image_string = image_labeler.image.title()
        image_data = io.imread(image_string)
        svg_size = 400
        # Rescalamos las coordenadas para que coincidan
        # las coordenadas de la vista con las de la imagen original
        real_image_h, real_image_w, _ = image_data.shape
        
        # Calculamos rescalado
#         coef_h = real_image_h / svg_size
#         coef_w = real_image_w / svg_size
        
        # TODO Cuidado con si es menor de  svg_size x svg_size
        
        # Calculamos padding en función de
        # los valores de ancho y alto de la imagen
        # en la vista
        if real_image_h > real_image_w:
            coef = real_image_h / svg_size
            svg_image_h = svg_size
            svg_image_w = round(real_image_w / coef)
        else:
            coef = real_image_w / svg_size
            svg_image_w = svg_size
            svg_image_h = round(real_image_h / coef)
        
        coef_h = real_image_h / svg_image_h
        coef_w = real_image_w / svg_image_w
#         print("svg image size: ",svg_image_h, svg_image_w)
#         print("real image size: ",real_image_h, real_image_w)
        
        svg_padding_h = (svg_size - svg_image_h) / 2
        svg_padding_w = (svg_size - svg_image_w) / 2
        
#         print("svg image padding: ",svg_padding_h, svg_padding_w)
#         print("coefs: ",coef_h, coef_w)
        
        # Por cada rectangulo generamos una nueva imagen
        for rect_y, rect_x, rect_width, rect_height in self._rectList:
            
#             print(rect_x,rect_x+rect_height,rect_y,rect_y+rect_width)
            
            rect_x = round((rect_x - svg_padding_h) * coef_h)
            rect_y = round((rect_y - svg_padding_w) * coef_w)
            rect_height = round(rect_height * coef_h)
            rect_width = round(rect_width * coef_w)
            #TODO comprobar que no se sale de la imagen
            
#             print(rect_x,rect_x+rect_height,rect_y,rect_y+rect_width)
            
            name = str(rect_x)+str(rect_y)+".jpg"
            img = image_data[rect_x:rect_x+rect_height,rect_y:rect_y+rect_width]
            
            io.imsave(name, img)

In [292]:
%%javascript
require.undef('ImageLabeler');

var startX, startY;
var count = 0;
var rect = null;
var rectsList = [];
var model;
var myThis;

function getMousePos(evt) {
    // Obtenemos posición absoluta del elemento svg
    var svgRef = document.getElementById("svg");
    var htmlPos = svgRef.getBoundingClientRect();
    return {
        x: evt.clientX - htmlPos.left,
        y: evt.clientY - htmlPos.top
    };
}

function updateRect(eve, rect){
    var pos = getMousePos(eve);
    var endX = pos.x;
    var endY = pos.y;
    if (endX - startX > 0){
        var x = startX
    }else{
        var x = endX;
    }

    if (endY - startY > 0){
        var y = startY
    }else{
        var y = endY;
    }
    var width = Math.abs(endX - startX);
    var height = Math.abs(endY - startY);
    
    rect.setAttributeNS(null, 'x', x);
    rect.setAttributeNS(null, 'y', y);
    rect.setAttributeNS(null, 'height', height);
    rect.setAttributeNS(null, 'width', width);
}

function setStyleRect(rect){
    rect.setAttributeNS(null, 'fill', 'transparent');
    rect.setAttributeNS(null, 'stroke', 'green');
    rect.setAttributeNS(null, 'linewidth', '10px');
}

function mouseMove(eve) {
    if(rect){
        updateRect(eve, rect);
      }
}

function mouseClick(eve) {
    if (rect !== null) {
        //Cada vez que finalizamos el rectangulo
        // , le añadimos a la lista
        console.log(rect.x['baseVal'].value,rect.y['baseVal'].value,rect.width['baseVal'].value,rect.height['baseVal'].value);
        rectsList = [];
        model.set('rect',[rect.x['baseVal'].value,rect.y['baseVal'].value,rect.width['baseVal'].value,rect.height['baseVal'].value]);
        myThis.touch();
        rect = null;
        console.log("finsihed.");
    } else {
        var pos = getMousePos(eve);
        
        startX= pos.x;
        startY = pos.y;
        
        console.log("begun.");
        
        if(!rect){
            rect = document.createElementNS("http://www.w3.org/2000/svg", 'rect');
            rect.id = 'rect' + count;
            count = count + 1;
            
            updateRect(eve, rect);

            setStyleRect(rect);
            document.getElementById('svg').appendChild(rect);
          }
        }
}

// Funciones para la carga y cambio de imagen
function loadImage(svg, src){
    svg.innerHTML = '<image id="image" xlink:href="'+ src +'" x="0" y="0" height="400px" width="400px"/>'
}

function changeImage(svg, el, image){
    console.log("Cambio de imagen");
    console.log(image);
    svg.innerHTML = '<image id="image" xlink:href="' + image + '" x="0" y="0" height="400px" width="400px"/>';
    el = svg
}


// Definición del Widget en javasript
define('ImageLabeler', ["jupyter-js-widgets"], function(widgets) {
    var svg;
    var ImageLabelerView = widgets.DOMWidgetView.extend({
        // Renderizar vista
        render: function() {
            model = this.model;
            myThis = this;
            // Creamos el SVG
            svg = document.createElementNS("http://www.w3.org/2000/svg", "svg");
            svg.id = 'svg';
            svg.setAttribute('width', '400px');
            svg.setAttribute('height', '400px');
            
            // Asignamos listeners al SVG
            svg.addEventListener("click", mouseClick, false);
            svg.addEventListener("mousemove", mouseMove, false);
            
            // Imágen
            var src = "../rsc/img/family.jpg";
            loadImage(svg, src);
            // Listener para cambios de imagen
            this.image_changed();
            this.model.on('change:image', this.image_changed, this);
            
            // Asignamos a la vista el elemento SVG
            this.el = svg
        },
        // Función que se encarga del cambio de imagen
         image_changed: function() {
             var image = this.model.get('image');
             changeImage(svg, this.el, image);
        },
    });

    return {
        ImageLabelerView: ImageLabelerView
    };
});

<IPython.core.display.Javascript object>

# Añadimos más widgets

En esta celda añadimos el *file upload*

In [293]:
from IPython.display import display
import fileupload
import PIL.Image
import io as io2
from skimage.color import rgb2gray

''' Función que se encarga de aplicar las operaciones 
necesarias para convertir los datos obtenidos del FileUpload
en una imagen'''
def imageConverter(change):
        ch = change['owner']
        image = io2.BytesIO(ch.data)
        image = PIL.Image.open(image)
        image = np.array(image)
        # Además, eliminamos la anterior 
        # y la guardamos
        rand = np.random.randint(0,1e6)
        #os.remove(rand + ".jpg")
        io.imsave(str(rand) + ".jpg",image)
        return str(rand) + ".jpg"

### Inicializamos el resto de Widgets

In [294]:
from ipywidgets import HBox, VBox, Label, Layout

#  Inicializamos el label título
title = Label(value = 'Etiquetador de imágenes')
#title.layout.align_items
title.layout.margin = '0 0 0 35%'
#print(title.layout)

#Inicializamos Widget de File Upload
upload_widget = fileupload.FileUploadWidget()
upload_widget.description = '(50% width, 80px height) button'

# Inicializamos Widget etiquetador de imágenes
image_labeler = ImageLabelerWidget()
# Callback para el cambio de imagen
def _cb(change):
    # Guardamos imagen
    image = imageConverter(change)
    # Sincronizamos cambio
    image_labeler.image = image

upload_widget.observe(_cb, names='data')

#
btns_selector =widgets.ToggleButtons(
    options=['Fitolito tipo 1', 
             'Fitolito tipo 2', 
             'Fitolito tipo 3',
             'Fitolito tipo 4',
             'Fitolito tipo 5',
             'Fitolito tipo 6',
             'Fitolito tipo 7',
             'Fitolito tipo 8'],
    #description='Tipo de fitolito',
    disabled=False,
    # 'success', 'info', 'warning', 'danger' or ''
    button_style='',
    tooltip='Description',
    #icon='check'
)
'''Función que se encarga de hacer los cambios 
necesarios al cambiar el fitolito que se etiqueta'''
def on_phytolith_change(change):
    new_dir = change['new']
#     print(new_dir)
    dirs_set.change_dir(new_dir)
    

# listener del cambio de fitolito
btns_selector.observe(on_phytolith_change,names='value')

# Formato de los widgets
right_size_widget = VBox([upload_widget, btns_selector])
right_size_widget.width = '10%'
right_size_widget.margin = '5% 5% 5% 5%'
btns_selector.padding = '10%'
upload_widget.padding = '10%'
image_labeler.margin = '5% 10% 0% 0%'
upload_widget.margin = '0 0 18% 12%'
upload_widget.width = 'auto'

In [295]:
HBox([image_labeler,right_size_widget])